In [1]:
import os
import json

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np

import s3fs
from datetime import datetime
from datetime import timedelta
import calendar

In [2]:
raw_test = pd.read_csv('s3://msds630-kaggle-competition/sessions_events_modified_v2.csv/part-00000', header=None)

In [3]:
columns = ["session_id", "start_timestamp","time_zone","timezone_offset","previous_sessions","user_created_stamp","is_first_session","country", "region","city","latitude","longitude","local","os_name","session_index","device_id","user_id_hash","num_events","num_purchases","weekend_purchase","is_purchase",'purchase_times','sum_purchase','1week','2week']
raw_test.columns = columns

In [4]:
import math

def convertTime(ts_str):
    if math.isnan(float(ts_str)):
        return None
    else:
        return datetime.utcfromtimestamp(int(ts_str)/1000.0).strftime('%Y-%m-%d %H:%M:%S')
convertTime('1541638424150')

'2018-11-08 00:53:44'

In [5]:
raw_test['start_timestamp'] = raw_test['start_timestamp'].map(lambda x: convertTime(x))
columns = ["user_id_hash","session_id", "start_timestamp","time_zone","timezone_offset","previous_sessions","user_created_stamp","is_first_session","country", "region","city","latitude","longitude","local","os_name","session_index","device_id","num_events","num_purchases","weekend_purchase","is_purchase",'purchase_times','sum_purchase','1week','2week']
raw_test = raw_test.reindex(columns,axis=1)

In [6]:
raw_test.head()

,user_id_hash,session_id,start_timestamp,time_zone,timezone_offset,previous_sessions,user_created_stamp,is_first_session,country,region,...,session_index,device_id,num_events,num_purchases,weekend_purchase,is_purchase,purchase_times,sum_purchase,1week,2week
0,9e095d1e1a4049b2c830d3884ca526c3cd00faadef7768...,6282115060269174490,2018-10-28 06:52:14,America/Los_Angeles,-25200000.0,127776870.0,1.539629e+12,False,US,wa,...,52.0,eaf0f10b-5e67-412a-9a30-7a687d902636,11.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0
1,965bbf8946615acfd53b9d4febc4bfb8f76d2ca8dcc47e...,3311962070546841600,2018-10-18 00:42:48,America/New_York,-14400000.0,0.0,1.539823e+12,True,US,nc,...,1.0,f90df33e-5bd3-41b0-af5e-3613bfaff7df,57.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0
2,8ef37a8bf91fcf32fd2dff7361140d062d25e89403498c...,2227925547996687280,2018-11-17 16:41:00,America/New_York,-18000000.0,148898.0,1.542405e+12,False,US,ny,...,3.0,779DA08D-B10E-45EC-89FB-C28D15E02C88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,8939252064915685380,2018-11-28 10:36:47,Australia/Sydney,39600000.0,61598736.0,1.540796e+12,False,AU,vic,...,58.0,f8ae3393-8a50-426c-8e61-b855478f4de0,4.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0
4,b8f647bd603c347e013739f5c3bb9568f21d6736c82fb9...,5558769389058741590,2018-11-13 20:07:28,Asia/Kolkata,19800000.0,17663056.0,1.541154e+12,False,IN,up,...,30.0,400E3C4E-55B6-4BAC-AEB9-12C0F9EB2FC6,42.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0


In [7]:
raw_test.drop(['time_zone', 'timezone_offset', "is_first_session", "region","city","latitude","longitude","local","session_index" ], axis=1, inplace=True)

In [8]:
raw_test.head()

,user_id_hash,session_id,start_timestamp,previous_sessions,user_created_stamp,country,os_name,device_id,num_events,num_purchases,weekend_purchase,is_purchase,purchase_times,sum_purchase,1week,2week
0,9e095d1e1a4049b2c830d3884ca526c3cd00faadef7768...,6282115060269174490,2018-10-28 06:52:14,127776870.0,1.539629e+12,US,Android OS,eaf0f10b-5e67-412a-9a30-7a687d902636,11.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0
1,965bbf8946615acfd53b9d4febc4bfb8f76d2ca8dcc47e...,3311962070546841600,2018-10-18 00:42:48,0.0,1.539823e+12,US,Android OS,f90df33e-5bd3-41b0-af5e-3613bfaff7df,57.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0
2,8ef37a8bf91fcf32fd2dff7361140d062d25e89403498c...,2227925547996687280,2018-11-17 16:41:00,148898.0,1.542405e+12,US,iOS,779DA08D-B10E-45EC-89FB-C28D15E02C88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,8939252064915685380,2018-11-28 10:36:47,61598736.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,4.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0
4,b8f647bd603c347e013739f5c3bb9568f21d6736c82fb9...,5558769389058741590,2018-11-13 20:07:28,17663056.0,1.541154e+12,IN,iOS,400E3C4E-55B6-4BAC-AEB9-12C0F9EB2FC6,42.0,0.0,NaN,-1.0,NaN,0.0,-1.0,-1.0


In [9]:
raw_train = raw_test[raw_test["start_timestamp"] < "2018-12-02"] 

In [10]:
raw_train.shape

(4583523, 16)

In [11]:
raw_test.shape

(5999419, 16)

In [12]:
one_user_sample = raw_train[raw_train['user_id_hash'] == raw_train.user_id_hash[3]]
one_user_sample

,user_id_hash,session_id,start_timestamp,previous_sessions,user_created_stamp,country,os_name,device_id,num_events,num_purchases,weekend_purchase,is_purchase,purchase_times,sum_purchase,1week,2week
3,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,8939252064915685380,2018-11-28 10:36:47,61598736.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,4.0,0.0,NaN,-1.0,NaN,0.000,-1.0,-1.0
195607,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,4945072353632227812,2018-11-21 21:45:03,38154412.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381696,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,3887088350881966374,2018-11-28 12:43:43,62229224.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,35.0,1.0,0.0,1.0,2018-11-28 12:52:14,3.493,-1.0,-1.0
420439,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,7928028454387268924,2018-11-22 12:04:36,39126024.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,78.0,1.0,0.0,1.0,2018-11-22 12:33:38,3.493,-1.0,-1.0
560350,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,7701405822190606406,2018-11-17 08:57:18,35233104.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,1.0,0.0,NaN,-1.0,NaN,0.000,-1.0,-1.0
790094,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,4685423420807294279,2018-11-29 12:23:51,64675772.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,53.0,0.0,NaN,-1.0,NaN,0.000,-1.0,-1.0
827695,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,4355843251754716269,2018-10-31 01:59:30,25500074.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,1.0,0.0,NaN,-1.0,NaN,0.000,-1.0,-1.0
862369,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,678537999137683660,2018-11-24 10:21:02,44690410.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,2.0,0.0,NaN,-1.0,NaN,0.000,-1.0,-1.0
1015333,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,8630810103643746631,2018-11-22 21:23:30,41878824.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053550,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,6417931150482904512,2018-12-01 07:16:38,67014810.0,1.540796e+12,AU,Android OS,f8ae3393-8a50-426c-8e61-b855478f4de0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
def user_features(one_user_sample):
    """
    user_id, num_sessions, mean_lenght_session, user_created, country_mode, os_mode, num_devices, max_events, sum_purchases, num_purchases, lifetime_value
    """
    features = []
    features.append(one_user_sample.user_id_hash.unique()[0])
    features.append(one_user_sample.session_id.count()) 
    features.append(int(one_user_sample.previous_sessions.mean()))
    features.append(one_user_sample.user_created_stamp.unique()[0])
    features.append(one_user_sample.country.mode()[0])
    features.append(one_user_sample.os_name.mode()[0])
    features.append(len(one_user_sample.device_id.unique()))
    features.append(one_user_sample.num_events.max())
    features.append(one_user_sample.num_purchases.sum())
    # features.append(one_user_sample.weekend_purchase.value_counts()[1]/float(one_user_sample.session_id.count()))
    features.append(one_user_sample.is_purchase.count())
    features.append(np.dot(one_user_sample.sum_purchase.value_counts().keys(), one_user_sample.sum_purchase.value_counts().values)/float(one_user_sample.session_id.count()))
    return features

In [62]:
users = raw_train.user_id_hash.unique()

In [ ]:
all_users = []

for user in users:
    all_users.append(user_features(raw_train[raw_train['user_id_hash'] == user]))

In [ ]:
df_train = pd.DataFrame(all_users)

In [ ]:
raw_train[raw_train['user_id_hash'] == raw_train.user_id_hash[3]]